## Arrhenius plot and diffusion kinetic parameters for step-heating experiments

This notebook uses the reduced JSON data file of each step-heating experiments from the HAL to create their Arrhenius plots and calculate the diffusion kinetic parameters.

In [ ]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#defining some constants
gas_constant = 8.314472   # universal gas constant in J/K*mol
avogadro = 6.022e+23   # in 1/mol

### 1. Input sample specific values

In the cell below, input all necessary information for the specific sample.

In [ ]:
Sample_ID = ''

# slab dimensions in microns
width = 
length_1 = 
length_2 = 

# insert zircon density
zirc_rho = 4.36  # in g/cm^3

# dose
dose_UPb =  # time-integrated dose in alphas/gram
dose_UTHe = 
dose_eq1 =   # equivalent dose (model 1) in alphas/gram
dose_eq2 =  # equivalent dose (model 1) in alphas/gram

half_width = (width/2) / 1e4 # in cm

In [ ]:
#calculating volume 

vol = (width * length_1 * length_2) / 1e12  # in cm^3

#calculating volumetric mol of He
#make sure to choose the alpha dose value you want to use

dose = 
He_mol_total = ((vol * zirc_rho) * dose) / avogadro   # in mol   

#vol_He = (1e15 * (vol * zirc_rho) * alpha) / avogadro  # in fmol

#vol_He_sci = "{:e}".format(vol_He)
#print(vol_He_sci)

print(He_mol_total, ' mol')

### 2. Import JSON file
Use the script below to read a JSON from a local file. Simply input the name of the JSON file as `filename.json`. IMPORTANT: the file needs to be in the same directory as this notebook.

In [ ]:
He_data = pd.read_json('_diff_cell_He_data.json', orient='index')

# adding column name to the respective columns
He_data.reset_index(inplace=True)  # assigning simple index for ease of calculation
He_data.columns =['sample_ID' , 'ncc_4He' , 'ncc_4He_1sig' , 'mol_He' , 'mol_He_1sig' , 'notes' , 'temp_degC', 'time_s', 't_err']

He_data

#### Appending temperature, time, and cumulative time

The easiest way would be to compile the heating temperature and time in an excel file (with column labels) and appending it to the data frame.

If temp and time were input in `He_line_calc_diff_cell.ipynb` script, then now just need to calculate `t_cum`.

In [ ]:
#adding new columns from excel
#temp_time = pd.read_excel('G168_heating_tT.xlsx')

#He_data['temp_degC'] = temp_time ['temp_degC'].values
#He_data['time_s'] = temp_time ['time_s'].values

#adding new column for cumulative time in s
He_data['t_cum'] = He_data['time_s'].cumsum()

He_data

### 3. Making an Arrhenius plot
#### Calculating the fraction released `frac`, cumulative fraction released `f_cum`, and ln(D/${a^2}$) `ln_D_a2` for all the heating steps.

In [ ]:
#empty lists that we'll append to
frac = []
ln_D_a2 = []

#calculating just the fraction released at individual steps

for i in range(len(He_data)):
    frac = He_data.mol_He / He_mol_total

He_data['frac'] = frac

#calculating cumulative fraction released and putting it into a new column
He_data['f_cum'] = He_data['frac'].cumsum()

# first define the first step with f_cum that is more or equal to 0.45
fcum_45 = 

#now calculating ln(D/a^2)
for i in range(len(He_data)):

    if i == 0:
        ln_Da2 = np.log((He_data.f_cum[i] ** 2) * (np.pi / (4 * He_data.time_s[i])))
    
    elif 0 < i < fcum_45:
        ln_Da2 = np.log((np.pi * (He_data.f_cum[i]**2 - He_data.f_cum[i-1]**2)) / (4 * He_data.time_s[i]))
        
    else:
        ln_Da2 = np.log((-4 / (np.pi**2 * He_data.time_s[i])) * np.log((1 - He_data.f_cum[i])/(1 - He_data.f_cum[i-1])))
        
    ln_D_a2.append(ln_Da2) 

#adding list into dataframe
He_data['ln_D_a2'] = ln_D_a2

He_data

#### Calculating uncertainties for `D_a2`.

We will use equations 20 and 24 of Ginster (2018, dissertation) to calculate the uncertainty for `D_a2`.

First, calculating error with respect to uncertainty in evolved gas quantities `mol_He_1sig`.

In [ ]:
D_a2_sig_X = []

#now calculating uncertainty for ln(D/a^2)
#but first define if you want to calc using 1 or 2 sig uncertainty of the evolved gas mol
sig = 1

for i in range(len(He_data)):

    if i == 0:
        long_eq = ((1 - He_data.f_cum[i]) * (sig*He_data.mol_He_1sig[i]))**2 + (He_data.f_cum[i]**2) \
                    * np.sum((sig*He_data.mol_He_1sig[1:])**2)
        Da2_sig_X = (np.pi/(2 * He_data.time_s[i])) * (He_data.f_cum[i]/He_mol_total) * np.sqrt(long_eq)
    
    elif 0 < i < fcum_45:
        long_eq1 = ((He_data.f_cum[i] * (1 - He_data.f_cum[i]) - He_data.f_cum[i-1] * (1 - He_data.f_cum[i-1]))**2) \
                    * np.sum((sig*He_data.mol_He_1sig[:i])**2)
        long_eq2 = ((He_data.f_cum[i] * (1 - He_data.f_cum[i]) + He_data.f_cum[i-1]**2)**2) \
                    * ((sig*He_data.mol_He_1sig[i])**2)
        long_eq3 = ((He_data.f_cum[i-1]**2 - He_data.f_cum[i]**2)**2) \
                    * np.sum((sig*He_data.mol_He_1sig[i+1:])**2)
        Da2_sig_X = (np.pi/(2 * He_data.time_s[i] * He_mol_total)) * np.sqrt(long_eq1 + long_eq2 + long_eq3)
        
    else:
        long_eq4 = ((1 + (He_data.f_cum[i-1] / (1 - He_data.f_cum[i-1])))**2) * (sig*He_data.mol_He_1sig[i])**2
        long_eq5 = ((He_data.f_cum[i-1]/(1 - He_data.f_cum[i-1]) - He_data.f_cum[i]/(1-He_data.f_cum[i]))**2) \
                    * np.sum((sig*He_data.mol_He_1sig[i+1:]**2))
        Da2_sig_X = (4 / (np.pi * He_data.time_s[i] * He_mol_total)) * np.sqrt(long_eq4 + long_eq5)
        
    D_a2_sig_X.append(Da2_sig_X) 

#adding list into dataframe
He_data['D_a2_sig_X'] = D_a2_sig_X

Next, we'll calculate error in `D_a2` wrt to `t_err`, `D_a2_sig_t`. For now we will use the cooling time as the `t_err`.

In [ ]:
D_a2_sig_t = []

#now calculating uncertainty for ln(D/a^2)

for i in range(len(He_data)):

    if i == 0:
        Da2_sig_t = np.absolute((np.pi * He_data.f_cum[i]**2 / 4) * (-1 * He_data.t_err[i] / He_data.time_s[i]**2))
    
    elif 0 < i < fcum_45:
        Da2_sig_t = np.absolute((np.pi * (He_data.f_cum[i]**2 - He_data.f_cum[i-1]**2) / 4) 
                                * (-1 * He_data.t_err[i] / He_data.time_s[i]**2))
        
    else:
        Da2_sig_t = np.absolute((-4 / np.pi**2 * np.log((1 - He_data.f_cum[i])/(1 - He_data.f_cum[i-1]))) 
                                * (-1 * He_data.t_err[i] / He_data.time_s[i]**2)) 
        
    D_a2_sig_t.append(Da2_sig_t) 

#adding list into dataframe
He_data['D_a2_sig_t'] = D_a2_sig_t

He_data

Now calculating error in D/a2 propogated from both `mol_He_1sig` and `t_err`, `D_a2_sig`, and calculating `ln_D_a2_sig`.

In [ ]:
D_a2_sig = []
ln_D_a2_sig = []

for i in range(len(He_data)):
    
    Da2_sig = np.sqrt(He_data.D_a2_sig_X[i]**2 + He_data.D_a2_sig_t[i]**2)
    ln_Da2_sig = np.log(np.exp(He_data.ln_D_a2[i]) + Da2_sig) - He_data.ln_D_a2[i]
    
    D_a2_sig.append(Da2_sig)
    ln_D_a2_sig.append(ln_Da2_sig)
    
He_data['D_a2_sig'] = D_a2_sig
He_data['ln_D_a2_sig'] = ln_D_a2_sig

He_data

#### Now plotting `ln_D_a2` vs 1e4/T(K)
Temperature is listed as degree C, so we'd need to convert that and divide it by 1e4. In order to be able to easily calculate linear regression and visualize linear vs non-linear trends, we will plot these separately. Therefore, the index number for the last NAB step should be changed for each experiment.

In [ ]:
plt.figure(figsize = (6,4), dpi=200)
plt.xlabel(r'$10^{4}$/T ($K^{-1}$)')
plt.ylabel(r'ln(D/$a^{2}$)')
plt.xlim(8, 26)
plt.ylim(-28, -8)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.title('G3 Arrhenius plot')

# FIRST OF ALL input the index for the FIRST Arrhenius datapoint that will be used for regression

arr_1 = 
arr_limit = 

# now we will make the Arrhenius plot. NOTE last step (index 20) is not included

x_axis1 = 1e4 / (He_data.temp_degC.iloc[:arr_1] + 273.15)
x_axis2 = 1e4 / (He_data.temp_degC.iloc[arr_1:arr_limit] + 273.15)
x_axis3 = 1e4 / (He_data.temp_degC.iloc[arr_limit:] + 273.15)
x_axis_btwn = 1e4 / (He_data.temp_degC.iloc[arr_1-1:arr_1+1] + 273.15)
x_axis_btwn2 = 1e4 / (He_data.temp_degC.iloc[arr_limit-1:arr_limit+1] + 273.15)

plt.plot(x_axis1, He_data.ln_D_a2.iloc[:arr_1], color = 'gray', linewidth=0.5)
plt.plot(x_axis2, He_data.ln_D_a2.iloc[arr_1:arr_limit], color = 'gray', linewidth=0.5)
plt.plot(x_axis3, He_data.ln_D_a2.iloc[arr_limit:], color = 'gray', linewidth=0.5)
plt.plot(x_axis_btwn, He_data.ln_D_a2.iloc[arr_1-1:arr_1+1], color = 'gray', linewidth=0.5)
plt.plot(x_axis_btwn2, He_data.ln_D_a2.iloc[arr_limit-1:arr_limit+1], color = 'gray', linewidth=0.5)
plt.scatter(x_axis1, He_data.ln_D_a2.iloc[:arr_1], color = 'b')
plt.scatter(x_axis2, He_data.ln_D_a2.iloc[arr_1:arr_limit], color = 'm')
plt.scatter(x_axis3, He_data.ln_D_a2.iloc[arr_limit:], color = 'r')

plt.errorbar(x_axis1, He_data.ln_D_a2.iloc[:arr_1], yerr=He_data.ln_D_a2_sig.iloc[:arr_1], fmt='o', capsize=6)
plt.errorbar(x_axis2, He_data.ln_D_a2.iloc[arr_1:arr_limit], yerr=He_data.ln_D_a2_sig.iloc[arr_1:arr_limit], fmt='o', capsize=6)
plt.errorbar(x_axis3, He_data.ln_D_a2.iloc[arr_limit:], yerr=He_data.ln_D_a2_sig.iloc[arr_limit:], fmt='o', capsize=6)

#plt.savefig('.pdf') 
plt.show()

### 4. Doing weighted and unweighted regression for Arrhennius plot
First we'll set up the linear regression equations

DON'T FORGET TO CHANGE THE n-value to the number of datapoints to regress!!!

In [ ]:
#THIS IS FOR UNWEIGHTED LINEAR REGRESSION
#insert n-value here. will be used for the rest of the calc
n = 

def ord_lin_regress(X_vals, Y_vals):
    
    #hint: it helps to create arrays for XY values and X^2 (use x**2 to raise something to a power) from your input
    #first create some empty arrays
    
    XY_vals = []
    X2_vals = []
    
    #then try this for loop (remember that Python uses zero indexing):
    
    for i in range(len(X_vals)):
        XY = X_vals[i] * Y_vals[i]
        X2 = X_vals[i]**2
        
        XY_vals.append(XY)
        X2_vals.append(X2)
        
    #now calculate slope and intercept below using the sum function, I'll let you try the math here
    
    slope = (sum(XY_vals) - (sum(X_vals)*sum(Y_vals)/n)) / (sum(X2_vals) - ((sum(X_vals)**2)/n))
    intercept = (sum(X_vals) * sum(XY_vals) - sum(Y_vals) * sum(X2_vals)) / ((sum(X_vals)**2) - n*sum(X2_vals))
    
    return slope, intercept

Now we'll calculate the slope and intercept for the Arrhenius trend. To make things easier we'll make a new dataframe only with the data we want to calculate the linear regression with.

In [ ]:
#THIS IS FOR UNWEIGHTED LINEAR REGRESSION

#making new lists. don't forget to change the range of datapoints
#first define the first index after the last data for regression, or basically the last index of data for regression plus 1
arr_limit = 

x_axis2 = 1e4 / (He_data.temp_degC.iloc[arr_1:arr_limit] + 273.15)
ln_D_a2_arr = He_data.ln_D_a2.iloc[arr_1:arr_limit]
ln_D_a2_sig_arr = He_data.ln_D_a2_sig.iloc[arr_1:arr_limit]

arrhenius_data = {'temp': x_axis2, 'ln_D_a2_arr':ln_D_a2_arr, 'ln_D_a2_sig_arr':ln_D_a2_sig_arr}
#making new dataframe and resetting index
arr_data = pd.DataFrame(arrhenius_data)
arr_data.reset_index(inplace=True)

#now we'll do the calculation
arrhenius = ord_lin_regress(arr_data.temp, arr_data.ln_D_a2_arr)

slope_un = arrhenius[0]
intercept_un = arrhenius[1]

print('The unweighted slope is ', arrhenius[0])
print('The unweighted intercept is ', arrhenius[1])

#### 4.5 Weighted linear regression with errors in y (`D_a2`)
Alternatively, we can calculate the slope and intercept using weighted linear regression, taking into account the errors in `ln_D_a2`.

In [ ]:
#first calculate the weighing factor, w that will be used for 
w_i = []

for i in range(len(arr_data)):
    w = (n * arr_data.ln_D_a2_sig_arr[i]**(-2)) / np.sum(arr_data.ln_D_a2_sig_arr**(-2))
    w_i.append(w)

arr_data['w_i'] = w_i

w_sum = np.sum(arr_data.w_i)
print(w_sum)  # check that this is equal to the number of datapoints to regress, n
arr_data

In [ ]:
wy_i = []
wx_i = []
wxy_i = []
w_x2_i = []  # w * x^2
    
#then try this for loop (remember that Python uses zero indexing):
    
for i in range(len(arr_data)):
    wy = arr_data.w_i[i] * arr_data.ln_D_a2_arr[i]
    wx = arr_data.w_i[i] * arr_data.temp[i]
    wxy = arr_data.w_i[i] * arr_data.temp[i] * arr_data.ln_D_a2_arr[i]
    w_x2 = arr_data.w_i[i] * arr_data.temp[i]**2
        
    wy_i.append(wy)
    wx_i.append(wx)
    wxy_i.append(wxy)
    w_x2_i.append(w_x2)
        
#now calculate slope and intercept below using the sum function, I'll let you try the math here
    
slope_wght = (n * sum(wxy_i) - sum(wx_i) * sum(wy_i)) / (n * sum(w_x2_i) - sum(wx_i)**2)
intercept_wght = (sum(wy_i) - slope_wght * sum(wx_i)) / n

print('The weighted slope is ', slope_wght)
print('The weighted intercept is ', intercept_wght)

### 5. Calculting std dev and 95% confidence interval in `slope` and `intercept` of linear regressions.
We will use the equation in "Chemometrics using R: Chapter 8.1".

#### Uncertainty for unweighted regression
First, we will calculate the standard deviation about the regression, `s_r`.

In [ ]:
y_min_y_2 =[]

#calculating std dev about the regression, s_r. uncomment the eq you want to use
for i in range(len(arr_data)):
    #reg_eq = slope_un * arr_data.temp[i] + intercept_un  # unweighted linear eq
    reg_eq = slope_wght * arr_data.temp[i] + intercept_wght  # weighted linear eq
    yi_min_yp_2 = (arr_data.ln_D_a2_arr[i] - reg_eq) ** 2
    
    y_min_y_2.append(yi_min_yp_2)
    
s_r = np.sqrt(np.sum(y_min_y_2) / (n - 2))  # same n as in section 4
print('Standard deviation about the regression =', s_r)

Now, we will calculate the standard deviations for the slope, `s_slope`, and y-intercept, `s_int`. IMPORTANT: Make sure to choose which equation to use for `reg_u` above!

In [ ]:
#first to calculate the mean of x (temp)
mean_T = np.mean(arr_data.temp)

x2 = []
x_min_x2 = []

# now to calculate x_i - x_expected in a for loop
for i in range(len(arr_data)):
    
    x2_i = arr_data.temp[i] ** 2
    x_min_x2_i = (arr_data.temp[i] - mean_T) ** 2
    
    x2.append(x2_i)
    x_min_x2.append(x_min_x2_i)
    
#calculating std dev for slope and intercept
s_slope = np.sqrt((s_r ** 2) / np.sum(x_min_x2))
s_int = np.sqrt((s_r**2 * sum(x2)) / (n * sum(x_min_x2)))

print(s_slope, s_int)

To calculate the confidence interval, we need to select the t-test critical values `alpha_t` and degrees of freedom `df` (simply n-2). Make sure to use the two-tailed `alpha_t`. The t value can be found online (e.g., https://www.medcalc.org/manual/t-distribution-table.php)

In [ ]:
# we want to calculate 95% confidence interval (alpha_T = 0.05)
t_test =   # for n =  (df = n - 2)

CI95_slope = s_slope * t_test
CI95_int = s_int * t_test

print(CI95_slope, CI95_int)

### 6. Calculate diffusion kinetic parameters  `E_a` and `D_0` with uncertainties.
Now we can calculate the diffusion kinetic parameters: activation energy `E_a` and frequency factor `D_0`.
#### NOTE: only either the weighted or unweighted uncertainties can be reported at a time due to the way the code is set up in part 5.

In [ ]:
#calculating activation energy and frequency factor with UNWEIGHTED lin reg

E_a = - gas_constant * slope_un * 10  # in kJ/mol
D_0 = np.exp(intercept_un) * half_width**2

#uncertainty calculation.

E_a_stdv = E_a - (- gas_constant * (slope_un+s_slope) * 10)
E_a_CI95 = E_a - (- gas_constant * (slope_un+CI95_slope) * 10)
D_0_stdev = np.exp(intercept_un+s_int) * half_width**2 - D_0
D_0_CI95 = np.exp(intercept_un+CI95_int) * half_width**2 - D_0

print(u'Unweighted E_a =', E_a, '\u00B1', E_a_stdv, 'kJ/mol (1\u03C3).')
print(u'Unweighted E_a =', E_a, '\u00B1', E_a_CI95, 'kJ/mol (95% CI).')
print(u'Unweighted D_0 =', D_0, '\u00B1', D_0_stdev, 'cm^2/s (1\u03C3).')
print(u'Unweighted D_0 =', D_0, '\u00B1', D_0_CI95, 'cm^2/s (95% CI).')
print('These are not the actual errors. Please see note for part 5 above.') #uncomment if necessary

In [ ]:
#calculating activation energy and frequency factor with WEIGHTED lin reg

Ea_w = - gas_constant * slope_wght * 10  # in kJ/mol
D0_w = np.exp(intercept_wght) * half_width**2

#uncertainty calculation.

Ea_w_stdv = Ea_w - (- gas_constant * (slope_wght+s_slope) * 10)
Ea_w_CI95 = Ea_w - (- gas_constant * (slope_wght+CI95_slope) * 10)
D0_w_stdev = np.exp(intercept_wght+s_int) * half_width**2 - D0_w
D0_w_CI95 = np.exp(intercept_wght+CI95_int) * half_width**2 - D0_w

print(u'Weighted E_a =', Ea_w, '\u00B1', Ea_w_stdv, 'kJ/mol (1\u03C3).')
print(u'Weighted E_a =', Ea_w, '\u00B1', Ea_w_CI95, 'kJ/mol (95% CI).')
print(u'Weighted D_0 =', D0_w, '\u00B1', D0_w_stdev, 'cm^2/s (1\u03C3).')
print(u'Weighted D_0 =', D0_w, '\u00B1', D0_w_CI95, 'cm^2/s (95% CI).')
#print('These are not the actual errors. Please see note for part 5 above.') #uncomment if necessary

### 7. Recalculating diffusion kinetic parameters `E_a` and `D_0` and their errors.
This time we'll use the compact equations 13(a-b) in York et al. (2004) to calculate the erros in the slope and intercept (and thus `E_a` and `D_0`. This is a little more elaborate than the method in section 5. We'll also take into account the errors in the x-axis `1e4/T`. We'll use the unweighted slope as the initial value of b.

In [ ]:
# making a new dataframe for this calculation
aliquot_ID = He_data.sample_ID.iloc[arr_1:arr_limit]

York_arr = pd.DataFrame()
York_arr['aliquot_ID'] = aliquot_ID
York_arr['temp'] = arr_data['temp'].values
York_arr['ln_D_a2_arr'] = arr_data['ln_D_a2_arr'].values
York_arr['ln_D_a2_sig_arr'] = arr_data['ln_D_a2_sig_arr'].values

# restarting index at 0 and deleting extra "Index" column
York_arr.reset_index(inplace=True)
York_arr = York_arr.drop(labels="index", axis=1)

# adding column for error in T from excel sheet (see spreadsheet for details)
temp_sig = pd.read_excel('G3_temp_err.xlsx')
York_arr.insert(loc = 3, column = 'temp_err', value = temp_sig.T_err)
#York_arr['temp_err'] = temp_sig['T_err'].values

In this next cell we'll calculate the weights for the X and Y vals (step 2), then do an iterative calculation for steps 3 - 5. This is done in a while loop (gay gods please help me).

In [ ]:
# first we'll define the slope, b and correlation `r_i`
slope_york = slope_un
r_i = 0  # no correlation between errors in x and y
    
# determining weights `w_x` and `w_y` and adding it to the dataframe

w_X = []
w_Y = []

for i in range(len(York_arr)):
    
    weight_X = 1 / (York_arr.temp_err[i])**2
    weight_Y = 1 / (York_arr.ln_D_a2_sig_arr[i])**2
    
    w_X.append(weight_X)
    w_Y.append(weight_Y)
    
York_arr['w_X'] = w_X
York_arr['w_Y'] = w_Y

# let's see if this works

tolerance = 1e-10  # input any number bigger than or equal to the desired tolerance
    
while tolerance >= 1e-15:
    
    W_i =[]
    U_i = []
    V_i = []
    beta_i = []    
    slope_guess = slope_york

    for i in range(len(York_arr)):
        alpha_i = np.sqrt(York_arr.w_X[i] * York_arr.w_Y[i])
        weight_W = (York_arr.w_X[i] * York_arr.w_Y[i]) / (York_arr.w_X[i] + (slope_guess**2) 
                                        * York_arr.w_Y[i] - (2 * slope_guess * r_i * alpha_i))
        W_i.append(weight_W)
        
    York_arr['W_i'] = W_i
        
    for i in range(len(York_arr)):
        
        X_mean = np.sum(York_arr.W_i * York_arr.temp) / np.sum(York_arr.W_i)
        Y_mean = np.sum(York_arr.W_i * York_arr.ln_D_a2_arr) / np.sum(York_arr.W_i)
    
        U_vals = York_arr.temp[i] - X_mean
        V_vals = York_arr.ln_D_a2_arr[i] - Y_mean
        
        beta_vals = York_arr.W_i[i] * (U_vals/York_arr.w_Y[i] + (slope_guess * V_vals)/York_arr.w_X[i] 
                            - (slope_guess*U_vals + V_vals) * (r_i / alpha_i))
        
        U_i.append(U_vals)
        V_i.append(V_vals)
        beta_i.append(beta_vals)
     
    York_arr['U_i'] = U_i
    York_arr['V_i'] = V_i
    York_arr['beta_i'] = beta_i
    
    slope_york = np.sum(York_arr.W_i * York_arr.beta_i * York_arr.V_i) / np.sum(York_arr.W_i * York_arr.beta_i * York_arr.U_i)
    tolerance = np.abs(slope_guess - slope_york)


York_arr['W_i'] = W_i   
York_arr['U_i'] = U_i
York_arr['V_i'] = V_i
York_arr['beta_i'] = beta_i

print('Slope = ', slope_york, slope_guess)
print('Tolerance = ', tolerance)

print('The weighted mean of X is', X_mean)
print('The weighted mean of Y is', Y_mean)
York_arr

In [ ]:
# now calculating the y-intercept `int_york`
intercept_york = Y_mean - slope_york * X_mean

# now calculating new E_a and D_0
Ea_york = - gas_constant * slope_york * 10  # in kJ/mol
D0_york = np.exp(intercept_york) * half_width**2

print('Intercept =', intercept_york)
print('E_a =', Ea_york, 'kJ/mol')
print('D_0 =', D0_york, 'cm^2/s')

In [ ]:
# calculatiing adjusted values `adj_x_i`
x_adj = []

for i in range(len(York_arr)):
    adj_x_i = X_mean + York_arr.beta_i[i]
    
    x_adj.append(adj_x_i)

York_arr['x_adj'] = x_adj

# calculating adjusted mean of x `mean_x_adj`
mean_x_adj = np.sum(York_arr.W_i * York_arr.x_adj) / np.sum(York_arr.W_i)

# calculating adjusted u `u_adj`
u_adj = []

for i in range(len(York_arr)):
    adj_u_i = York_arr.x_adj[i] - mean_x_adj
    
    u_adj.append(adj_u_i)
    
York_arr['u_adj'] = u_adj

# calculating error in slope `sig_slope_york` and intercept `sig_int_york`
sig_slope_york = np.sqrt(1 / np.sum(York_arr.W_i * (York_arr.u_adj**2)))
sig_int_york = np.sqrt((1 / np.sum(York_arr.W_i)) + (mean_x_adj**2) * (sig_slope_york**2))

# finally calculating error in `Ea_york` and `D0_york`
Ea_york_sig = Ea_york - (- gas_constant * (slope_york+sig_slope_york) * 10)
D0_york_sig_max = (np.exp(intercept_york+sig_int_york) * half_width**2) - D0_york
D0_york_sig_min = D0_york - (np.exp(intercept_york-sig_int_york) * half_width**2)

print(dose, 'alphas/g')
print(u'Weighted (with EVA) E_a =', Ea_york, '\u00B1', Ea_york_sig, 'kJ/mol (1\u03C3).')
print(u'Weighted (with EVA) D_0 =', D0_york, '\u00B1', '(-', D0_york_sig_min, ', +',  D0_york_sig_max, ') cm^2/s (1\u03C3).')

### 8. Arrhenius plot with trend
Here we will replot the Arrhenius plot with the unweighted/weighted linear reg trend.

In [ ]:
plt.figure(figsize = (6, 5), dpi=200)
plt.xlabel(r'$10^{4}$/T ($K^{-1}$)')
plt.ylabel(r'ln(D/$a^{2}$)')
plt.xlim(12, 26)
plt.ylim(-28, -10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
#plt.title('G3 Arrhenius plot')
plt.rcParams['axes.linewidth'] = 0.9

# now we will make the Arrhenius plot. NOTE last step (index 20) is not included

x_axis1 = 1e4 / (He_data.temp_degC.iloc[:arr_1] + 273.15)
x_axis2 = 1e4 / (He_data.temp_degC.iloc[arr_1:arr_limit] + 273.15)
x_axis_btwn = 1e4 / (He_data.temp_degC.iloc[arr_1-1:arr_1+1] + 273.15)
x_regtrend = np.arange(12,28) # defining range of x for regression trend

plt.plot(x_axis1, He_data.ln_D_a2.iloc[:arr_1], color = 'gray', linewidth=0.75)
plt.plot(x_axis2, He_data.ln_D_a2.iloc[arr_1:arr_limit], color = 'gray', linewidth=0.75)
plt.plot(x_axis_btwn, He_data.ln_D_a2.iloc[arr_1-1:arr_1+1], color = 'gray', linewidth=0.75)

# overlapping bigger datapoints
#plt.scatter(x_axis1, He_data.ln_D_a2.iloc[:arr_1], color = 'b')
#plt.scatter(x_axis2, He_data.ln_D_a2.iloc[arr_1:], color = 'm')

# plotting error bars with option to magnify errors by f_err degrees
f_err = 

plt.errorbar(x_axis1, He_data.ln_D_a2.iloc[:arr_1], yerr=f_err*He_data.ln_D_a2_sig.iloc[:arr_1], fmt='o', alpha=1,
             markeredgecolor='royalblue', markerfacecolor='white',markersize=12, color='royalblue', capsize=6)
plt.errorbar(x_axis2, He_data.ln_D_a2.iloc[arr_1:arr_limit], yerr=f_err*He_data.ln_D_a2_sig.iloc[arr_1:arr_limit], 
             markeredgecolor='brown', markerfacecolor='firebrick', markersize=12, color='brown', alpha=0.75,
             xerr=York_arr.temp_err, fmt='o', capsize=6)

# finally plotting
reg_wght = slope_wght * x_regtrend + intercept_wght
reg_unwght = slope_un * x_regtrend + intercept_un
reg_york = slope_york * x_regtrend + intercept_york
#plt.plot(x_regtrend, reg_wght, color = 'blue')
#plt.plot(x_regtrend, reg_unwght, color = 'green')
plt.plot(x_regtrend, reg_york, color = 'darkslategray', linestyle = '--', zorder=5, alpha=0.6)

# putting text and assigning values in the desired form
def as_si(x, ndp):
    s = '{x:0.{ndp:d}e}'.format(x=x, ndp=ndp)
    m, e = s.split('e')
    return r'{m:s}\times 10^{{{e:d}}}'.format(m=m, e=int(e))

textstr = '\n'.join((
    'Sample G3',
    r'$\alpha$-dose = ${0:s}$'.format(as_si(dose_eq2,2)),
    '$\mathrm{E_a}=%.2f$'%(Ea_york),
    'D$_0$ = ${0:s}$'.format(as_si(D0_york,2))))

plt.text(12.6,-27,textstr,fontsize=12)

plt.savefig('G3_Arrhenius_plot.pdf', transparent=True)
plt.show()

### 9. Calculating chi-squared and reduced chi-squared
This is a simple test to evaluate the goodness of fit of the regression. First, we'll calculate the `chi_sq` and then we'll calculate the `red_chisq`. This method takes into account each datapoint's error, and assumes that the errors are Gaussian. Basically we are maximizing the likelihood function.

To assess reduced chi-squared:
1. If `red_chisq` > 1, it is considered a "bad" fit
2. If `red_chisq` < 1, it is considered an overfit
3. The closer `red_chisq` is to 1, the better the fit

Additionally, we can also "improve" the reduced chi-squared (to 1) by calibrating the errors, but this is a problem for future me in part 9 of this very lengthy notebook.

In [ ]:
chisq = [] # making a list of individual terms

# note that `reg_eq` is either the weighted/unweighted function. see part 5

for i in range(len(arr_data)):
    unwght_reg = slope_un * arr_data.temp[i] + intercept_un  # unweighted reg eq
    wght_reg = slope_wght * arr_data.temp[i] + intercept_wght  # weighter reg eq
    chi = ((arr_data.ln_D_a2_arr[i] - wght_reg)**2 / (f_err*arr_data.ln_D_a2_sig_arr[i])**2)
    
    chisq.append(chi)
    
# calculating chi-squared
chi_sq = np.sum(chisq)

# now calculating reduced chi-squared, red_chisq
red_chisq = chi_sq / (n - 2)

red_chisq

Calculating reduced chi squared for the weighted least square solution with EVA (ie, following York et al. (2004))

In [ ]:
# revise this to account for error in x

chisq_york = [] # making a list of individual terms

for i in range(len(York_arr)):
    york_reg = slope_york * York_arr.temp[i] + intercept_york  # weighted reg eq
    chi_york = ((York_arr.ln_D_a2_arr[i] - york_reg)**2 / (f_err*York_arr.ln_D_a2_sig_arr[i])**2)
    
    chisq_york.append(chi_york)
    
# calculating chi-squared
chi_sq_york = np.sum(chisq_york)

# now calculating reduced chi-squared, red_chisq
red_chisq_york = chi_sq_york / (n - 2)

red_chisq_york

### 10. Exporting data

In [ ]:
# exporting dataframe as an excel file
He_data.to_excel("_Arr_data.xlsx")

### END (FOR NOW)
misc stuff below

Some random but useful lines of code:

In [ ]:
#adding new empty columns
#He_data.insert(loc=4, column='temp_degC', value='')

#appending new values one by one OR to replace an existing one
#He_data.at['RS23_G168_200_1', 'temp_degC'] = 200
#He_data.at['RS23_G168_200_1', 'time_s'] = 3600

#if you need to delete a row/column for some reason
#He_data = He_data.drop(labels="time_s", axis=1)

Calculating fraction and ln(D/${a^2}$) for the first heating step (not needed anymore but just keeping it here)

In [ ]:
#calculating first fraction, f_first and ln(D/a^2)

#f_first = He_data.mol_He[0] / He_mol_total
#ln_D_a2 = np.log((f_first**2) * (np.pi / (4 * He_data.time_s[0])))

#print(f_first, ln_D_a2)

#adding columns for frac, f_cum, and ln_D_a2
#He_data['frac'] = ''
#He_data['f_cum'] = ''
#He_data['ln_D_a2'] = ''

#appending to first row of dataframe

#He_data.at['RS23_G168_150_1', 'frac'] = f_first
#He_data.at['RS23_G168_150_1', 'ln_D_a2'] = ln_D_a2

#He_data